In [115]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import spacy as sp
import rich as rc
import os

In [116]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=os.getenv("OPENAI_API_TOKEN"),
)

In [117]:
with open("../data/introduccion.txt", "r", encoding="utf-8") as file:
    text = file.read()

for sentence in text.split(".")[:3]:
    rc.print(sentence.strip() + ".")

Aunque su verdadero nombre era Cosme Pérez, fue conocido artísticamente como 'Juan Rana', según documenta ya la 
Genealogía  —este apodo ya se le aplicaba en 1636—.

Aunque Cotarelo aventuró que pudo haber nacido en Madrid , hoy sabemos que nació en Tudela de Duero (Valladolid), 
en cuya iglesia parroquial fue bautizado el 7 de abril de 1593.

Fue hijo de Damián Pérez e Isabel de Basto, su segunda esposa, y fue el cuarto de una familia de cuatro hermanos.

In [118]:
nlp = sp.load("es_core_news_lg")
doc = nlp(text)

In [119]:
from collections import defaultdict

prompting_ners = defaultdict(set)
spacy_ners = defaultdict(set)

In [130]:
ner_extractor_prompt = PromptTemplate(
    input_variables=["text"],
    template="""
    Extrae todas las entidades nombradas que aparezcan en el siguiente texto, sin limitarte a ningún tipo predefinido. Por cada entidad encontrada, indícala en el formato:

    TIPO_DE_NER-NER

    El tipo debe estar en mayúsculas, en ingles, y representar con claridad a qué categoría pertenece cada entidad (como PER, ORG, GPE, LOC, DATE, TIME, MONEY, PERCENT, PRODUCT, EVENT, WORK_OF_ART, LAW, LANGUAGE, etc.). Si el tipo no está claro, utiliza la mejor etiqueta posible.

    Ejemplo de formato (solo como referencia, no restrictivo):

    ### Ejemplo:

    Texto:
    "Gabriel García Márquez escribió Cien años de soledad en Colombia en 1967."

    Salida:
    PER;Gabriel García Márquez
    WORK_OF_ART;Cien años de soledad
    LOC;Colombia
    DATE;1967

    No añadas explicaciones ni comentarios. Solo devuelve una lista, una entidad por línea, en el formato indicado.

    ### Texto:
    {text}
    
    ### Salida:
    """,
)

In [131]:
llm_response = llm.invoke(ner_extractor_prompt.format(text=text)).content
llm_response

'PER;Cosme Pérez  \nPER;Juan Rana  \nORG;Genealogía  \nDATE;1636  \nGPE;Madrid  \nGPE;Tudela de Duero  \nGPE;Valladolid  \nDATE;7 de abril de 1593  \nPER;Damián Pérez  \nPER;Isabel de Basto  \nPER;Sánchez Arjona  \nPER;Subirá  \nPER;Bernarda Ramírez  \nPER;Rennert  \nPER;Bernarda Manuela  \nPER;Cotarelo  \nPER;María de Acosta  \nPER;Francisca María Pérez  \nORG;Cofradía de Nuestra Señora de la Novena  \nPER;Tomás Fernández  \nDATE;1631  \nPER;Domingo Canejil  \nDATE;1707  \nPER;Bárbara Coronel  \nGPE;Guadalajara  \nPER;M. L. Lobato  \nWORK_OF_ART;La portería de las damas  \nPER;Avellaneda  \nPER;Pedro de la Rosa  \nPER;María Teresa de Austria  \nPER;H. Bergman  \nLOC;Madrid  \nLOC;Salón del Palacio  \nLOC;Bello Retiro  \nPER;Luis de Belmonte  \nWORK_OF_ART;La maestra de gracias  \nPER;Luis Quiñones de Benavente  \nWORK_OF_ART;El poeta de bailes  \nWORK_OF_ART;El doctor Juan Rana  \nDATE;1636  \nPER;F. Serralta  \nWORK_OF_ART;La risa y el actor: el caso de Juan Rana  \nPER;I. Arellano  

In [132]:
llm_response.split("\n")

['PER;Cosme Pérez  ',
 'PER;Juan Rana  ',
 'ORG;Genealogía  ',
 'DATE;1636  ',
 'GPE;Madrid  ',
 'GPE;Tudela de Duero  ',
 'GPE;Valladolid  ',
 'DATE;7 de abril de 1593  ',
 'PER;Damián Pérez  ',
 'PER;Isabel de Basto  ',
 'PER;Sánchez Arjona  ',
 'PER;Subirá  ',
 'PER;Bernarda Ramírez  ',
 'PER;Rennert  ',
 'PER;Bernarda Manuela  ',
 'PER;Cotarelo  ',
 'PER;María de Acosta  ',
 'PER;Francisca María Pérez  ',
 'ORG;Cofradía de Nuestra Señora de la Novena  ',
 'PER;Tomás Fernández  ',
 'DATE;1631  ',
 'PER;Domingo Canejil  ',
 'DATE;1707  ',
 'PER;Bárbara Coronel  ',
 'GPE;Guadalajara  ',
 'PER;M. L. Lobato  ',
 'WORK_OF_ART;La portería de las damas  ',
 'PER;Avellaneda  ',
 'PER;Pedro de la Rosa  ',
 'PER;María Teresa de Austria  ',
 'PER;H. Bergman  ',
 'LOC;Madrid  ',
 'LOC;Salón del Palacio  ',
 'LOC;Bello Retiro  ',
 'PER;Luis de Belmonte  ',
 'WORK_OF_ART;La maestra de gracias  ',
 'PER;Luis Quiñones de Benavente  ',
 'WORK_OF_ART;El poeta de bailes  ',
 'WORK_OF_ART;El doctor Jua

In [133]:
for item in llm_response.split("\n"):
    try:
        ner, name = item.split(";")
    except ValueError:
        print(f"Error: {item}")
        continue
    prompting_ners[ner].add(name.strip())

In [134]:
prompting_ners

defaultdict(set,
            {'PER': {'Agustín Moreto',
              'Alonso de Olmedo',
              'Ana Coronel',
              'Andrés de Claramonte',
              'Antonio de Escamilla',
              'Antonio de Solís',
              'Avellaneda',
              'Bartola',
              'Bergman',
              'Bernarda Manuela',
              'Bernarda Ramírez',
              'Bárbara Coronel',
              'Calderón',
              'Christiane Faliu-Lacourt',
              'Cosme Pérez',
              'Cotarelo',
              'D. D. Miller',
              'Damián Pérez',
              'Domingo Canejil',
              'E. H. Friedman',
              'El Capitán Medrano',
              'F. Serralta',
              'F. Sáez Raposo',
              'Felipe II',
              'Felipe IV',
              'Felipe Próspero',
              'Francisca María Pérez',
              'H. Bergman',
              'H. J. Manzari',
              'I. Arellano',
              'Isabel de Basto',


In [125]:
prompting_ners.keys()

dict_keys(['PER', 'ORG', 'DATE', 'GPE', 'WORK_OF_ART', 'LOC', 'EVENT'])

In [126]:
for ent in doc.ents:
    spacy_ners[ent.label_].add(ent.text)

In [127]:
spacy_ners

defaultdict(set,
            {'PER': {'Agustín Moreto',
              'Alcayde',
              'Alonso de] Olmedo',
              'Ana',
              'Ana Coronel',
              'Andrés de Claramonte',
              'Antonio Escamilla',
              'Antonio de Escamilla',
              'Antonio de Solís',
              'Antonio de] Escamilla',
              'Arellano',
              'Avellaneda',
              'Años',
              'Bartola',
              'Bergman',
              'Bernarda Manuela',
              'Bernarda Ramírez',
              'Borja',
              'Bárbara Coronel',
              'Cabredo',
              'Calderón',
              'Calderón Fieras',
              'Calderón de la Barca',
              'Capitán Medrano',
              'Cid Campeador',
              'Como',
              'Cosme',
              'Cosme Pérez',
              'Cosme [Pérez',
              'Cotarelo',
              'D. D. Miller',
              'Damián Pérez',
              'Doctor Ju

In [128]:
spacy_ners.keys()

dict_keys(['PER', 'MISC', 'LOC', 'ORG'])

In [135]:
print("Entidades encontradas por el modelo de lenguaje:")
for ner_type in prompting_ners.keys():
    print(f"\t- {ner_type}: {len(prompting_ners[ner_type])} entidades")

print("\nEntidades encontradas por el modelo de Spacy:")
for ner_type in spacy_ners.keys():
    print(f"\t- {ner_type}: {len(spacy_ners[ner_type])} entidades")

Entidades encontradas por el modelo de lenguaje:
	- PER: 68 entidades
	- ORG: 6 entidades
	- DATE: 34 entidades
	- GPE: 9 entidades
	- WORK_OF_ART: 40 entidades
	- LOC: 7 entidades
	- EVENT: 2 entidades

Entidades encontradas por el modelo de Spacy:
	- PER: 107 entidades
	- MISC: 65 entidades
	- LOC: 36 entidades
	- ORG: 2 entidades


Entidades encontradas por el modelo de lenguaje:
	- PER: 64 entidades
	- ORG: 8 entidades
	- DATE: 33 entidades
	- GPE: 6 entidades
	- WORK_OF_ART: 33 entidades
	- LOC: 6 entidades
	- EVENT: 2 entidades

Entidades encontradas por el modelo de Spacy:
	- PER: 107 entidades
	- MISC: 65 entidades
	- LOC: 36 entidades
	- ORG: 2 entidades

In [22]:
spacy_ners["LOC"]

{'Aguilar Priego',
 'Alcalde',
 'Arte de la pintura',
 'Bello',
 'Biblioteca Nacional de Madrid',
 'Buen Retiro',
 'Buen) Retiro',
 'Cantarranas',
 'Cofradía de Nuestra Señora de la Novena',
 'Coronela',
 'Escamilla',
 'España',
 'Españolas',
 'Espinel',
 'Grandes',
 'Grifona',
 'Guadalajara',
 'Infantes',
 'Kassel',
 'Lisboa',
 'Londres',
 'Madrid',
 'New Orleans',
 'Palacio',
 'Palacio de la Zarzuela',
 'Ranilla',
 'Reichenberger',
 'Retiro',
 'Salta',
 'Salón',
 'Salón del Palacio',
 'Salón del Palacio de Buen Retiro',
 'Subirá',
 'Tudela de Duero',
 'Valladolid',
 'los Reyes'}

In [23]:
prompting_ners["WORK_OF_ART"]

{'A la fiesta que hizo en el Retiro a los Reyes el Príncipe de Astillano',
 'Aguardad, supremos dioses',
 'Al cabo de los bailes mil',
 'Bien vengas mal',
 'El ayo de Agustín Moreto',
 'El caballero novel',
 'El doctor Juan Rana',
 'El guardainfante',
 'El mago',
 'El mundo',
 'El mundo al revés',
 'El niño caballero',
 'El poeta de bailes',
 'El poeta de bailes y el letrado. Segunda parte',
 'El remediador',
 'El retrato de Juan Rana',
 'El segundo Séneca de España',
 'El soldado',
 'El triunfo de Juan Rana',
 'Fieras afemina amor',
 'Juan Rana, a Gay Golden Age Gracioso',
 'La Zarzuela',
 'La infelice Dorotea',
 'La loa de Juan Rana',
 'La maestra de gracias',
 'La portería de las damas',
 'Las fiestas bacanales',
 'Lo que ha de ser',
 'Los muertos vivos',
 'Los volatines',
 'Pipote en nombre de Juan Rana',
 'Primus calamus',
 'Salta en banco',
 'Triunfos de Amor y Fortuna'}

In [136]:
import json

with open("../out/prompting_ners_gpt_4.1_mini.json", "w", encoding="utf-8") as fw:
    json.dump({key: list(values) for key, values in prompting_ners.items()}, fw, ensure_ascii=False, indent=4)
    
with open("../out/spacy_ners.json", "w", encoding="utf-8") as fw:
    json.dump({key: list(values) for key, values in spacy_ners.items()}, fw, ensure_ascii=False, indent=4)